In [57]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Memory hierarchy

## How much time did I spent on "memory"

Remember our demo long time ago?

In [ ]:
compare([do_render_code("./madd/madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("./madd/madd_B.c", lang="c++", show=["//START", "//END"])])

In [ ]:
render_code("./mv/matvec.c", lang="c++", show=["//I_FIRST_START", "//I_FIRST_END"])

In [ ]:
! cd mv; make clean; make; ./matvec 16 1 1 >> mv_addresses.csv ; head -n 100 mv_addresses.csv

In [ ]:
! cd mv; ./matvec 16 1 0 >> mv_addresses_j_first.csv ; head -n 100 mv_addresses_j_first.csv

## The memory hierarchy of my computer

In [ ]:
# Hung-Wei's Desktop
! lscpu | grep "Model name"
! getconf -a | grep CACHE

In [ ]:
# Your CS203 Cluster
! cs203 job run "lscpu | grep 'Model name'; getconf -a | grep CACHE"

In [ ]:
! ssh htseng@nano "lscpu | grep 'Model name'; getconf -a | grep CACHE"

## How blocks are stored in a cache?

How these address are stored "if" they're in a direct-mapped, 16B-sized blocks, 16-block cache?

In [ ]:
! cd mv; make clean; make; ./matvec 16 1 

In [ ]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
index_bits = int(math.log2(number_of_blocks))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

What if we have a 2-way, 16-byte blocked, 16-block cache?

In [ ]:
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
way_assoc=2
number_of_sets = number_of_blocks/way_assoc
index_bits = int(math.log2(number_of_sets))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

## Cache performance of code on "real machines"

### NVIDIA Jetson Nano -- Tegra X1

In [58]:
render_code("4way_madd/madd.c", show=["//START","//END"])

// 4way_madd/madd.c:51-63 (13 lines)
  //START
    for(i = 0; i < RUN_ARRAY_SIZE; i++)
    {
        e[i] = (a[i] * b[i] + c[i])/d[i];
#ifdef DUMP
        fprintf(stderr, "a[%d], %p\n",i,&a[i]);
        fprintf(stderr, "b[%d], %p\n",i,&b[i]);
        fprintf(stderr, "c[%d], %p\n",i,&c[i]);
        fprintf(stderr, "d[%d], %p\n",i,&d[i]);
        fprintf(stderr, "e[%d], %p\n",i,&e[i]);
#endif
    }
  //END

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

#### Run without the 4-way matrix add loop code.

In [59]:
# Run it "without" the above code.
! ssh htseng@nano "lscpu; cd courses/CS203/demo/memory/4way_madd/; make clean madd_nano; valgrind --tool=cachegrind ./madd_nano 16384 0 "

Architecture:        aarch64
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  1
Core(s) per socket:  4
Socket(s):           1
Vendor ID:           ARM
Model:               1
Model name:          Cortex-A57
Stepping:            r1p1
CPU max MHz:         1479.0000
CPU min MHz:         102.0000
BogoMIPS:            38.40
L1d cache:           32K
L1i cache:           48K
L2 cache:            2048K
Flags:               fp asimd evtstrm aes pmull sha1 sha2 crc32
rm -f madd_intel madd_nano *_O3 *~ madd_A_fission cachegrind* *.perf madd_dump
cc -O3 -DHAVE_LINUX_PERF_EVENT_H -g -DNANO madd.c -o madd_nano
==27725== Cachegrind, a cache and branch-prediction profiler
==27725== Copyright (C) 2002-2017, and GNU GPL'd, by Nicholas Nethercote et al.
==27725== Using Valgrind-3.13.0 and LibVEX; rerun with -h for copyright info
==27725== Command: ./madd_nano 16384 0
==27725== 
--27725-- Warning: Cannot auto-detect cache config, using defaults.
--2772

Too much detail! Let's use grep to narrow down the outputs.

In [60]:
# Run it "without" the above code.
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_nano 8192 0 >& nano_without_loop.perf; grep 'D   refs\|D1' nano_without_loop.perf"

==28165== D   refs:      1,527,706  (941,636 rd   + 586,070 wr)
==28165== D1  misses:        8,543  (  2,744 rd   +   5,799 wr)
==28165== D1  miss rate:       0.6% (    0.3%     +     1.0%  )


Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [61]:
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/;valgrind --tool=cachegrind ./madd_nano 8192 8192 >& nano_with_loop.perf; grep 'D   refs\|D1' nano_with_loop.perf"

==28320== D   refs:      1,548,192  (958,025 rd   + 590,167 wr)
==28320== D1  misses:       29,036  ( 19,138 rd   +   9,898 wr)
==28320== D1  miss rate:       1.9% (    2.0%     +     1.7%  )


In [ ]:
# Let's do some math here
total_number_of_accesses_before_the_loop =  1527706
total_number_of_accesses_after_the_loop = 1,548192
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 8530
total_number_of_misses_after_the_loop = 29021
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 4
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

### AMD RyZen 5 5500 -- 8-way L1, 64B-blocked, 32KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
! echo "element,address"> addresses_madd.csv; 
! cs203 job run "./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv"
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

#### Run without the 4-way matrix add loop code.

In [ ]:
# Run it "without" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/
! cs203 job run "lscpu; rm ./4way_madd/madd_intel; make -C 4way_madd madd_intel; valgrind --tool=cachegrind ./4way_madd/madd_intel 16384 0 2>> amd_without_loop.perf"
! grep 'D   refs\|D1' amd_without_loop.perf

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [ ]:
# Run it "with" the above code.
! cs203 job run "valgrind --tool=cachegrind ./4way_madd/madd_intel 16384 16384  2>> amd_with_loop.perf"
! grep 'D   refs\|D1' amd_with_loop.perf

In [ ]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2289801
total_number_of_accesses_after_the_loop = 2330796
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13544
total_number_of_misses_after_the_loop = 23789
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

### Intel Core i7 12700K -- 12-way L1, 64B-blocked, 48KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

#### Run without the 4-way matrix add loop code.

In [ ]:
# Run it "without" the above code.
! lscpu; cd ~/courses/CS203/demo/memory/4way_madd/; rm madd_intel; make madd_intel; valgrind --tool=cachegrind ./madd_intel 16384 0 >& intel_without_loop.perf; grep 'D   refs\|D1' intel_without_loop.perf

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [ ]:
# Run it "with" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf

In [ ]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2285911
total_number_of_accesses_after_the_loop = 2326877
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13447
total_number_of_misses_after_the_loop = 23694
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [ ]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

Now, let's take a look of what's happening!

In [ ]:
! cd prefetch; make clean; make
! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"

In [ ]:
! echo "With prefetch"; ssh htseng@celebi " cd courses/CS203/demo/memory/prefetch/; ./transpose_prefetch"


Let's try a different machine now.


In [ ]:
! cd prefetch; make clean; make; lscpu | grep Model
! echo "Without prefetch -- the baseline"; cd prefetch/; ./transpose
! echo "With prefetch"; cd prefetch/; ./transpose_prefetch

-- It doesn't work always!